# 세포 전자현미경을 통한 세포 이미지의 segmentation w/UNET 학습 및 테스트
* isbi 2012 EM (Electron Microscopy) dataset과 UNET을 이용한 segmentation.
* 해당 jupyter 포함내용
  * train, evaluation (single GPU or multi GPUs 선택가능)
  * visualization sample
  * output save & visualization
  * jupyter tensorboard

In [ ]:
import os
import numpy as np
import time
import pandas as pd


import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter    # tensorboard pip 설치시 이용가능

from model import UNet
from dataset import *
from util import *

import matplotlib.pyplot as plt

from torchvision import transforms, datasets


# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [ ]:
# GPU 이름 체크(cuda:0에 연결된 그래픽 카드 기준)
# print((torch.cuda.get_device_name(), device = 0)) # 'NVIDIA TITAN X (Pascal)'

# 사용 가능 GPU 개수 체크
# print(torch.cuda.device_count()) # 3


# gpu_count = torch.cuda.device_count()

# if gpu_count > 0:
#     print(f"현재 사용 가능한 GPU 수: {gpu_count}")
#     for i in range(gpu_count):
#         # 각 GPU의 번호와 이름을 확인합니다.
#         gpu_name = torch.cuda.get_device_name(i)
#         print(f"GPU {i}: {gpu_name}")
# else:
#     print("사용 가능한 GPU가 없습니다.")

## st1. Setting custom

In [ ]:
lr = 1e-2
batch_size = 5
num_epoch = 60 # 300  

data_dir = "datasets"
ckpt_dir = "out/checkpoint"
log_dir = "out/log"
result_dir = "out/result"

mode = "train"    # train or test
train_continue = "off"
use_gpu = True
multi_gpu = True
gpu_idx = "0"    # GPU 환경일 때 이용할 GPU설정 (GPU 환경 아닐 경우 사용안함)

## st2. environment setting & dataset loading
* 231011 - gpu idx selecting issue
* [CUDA_VISIBLE_DEVICES is not working 링크](https://discuss.pytorch.org/t/os-environ-cuda-visible-devices-not-functioning/105545/2) The CUDA_VISIBLE_DEVICES environment variable is read by the cuda driver. So it needs to be set before the cuda driver is initialized. It is best if you make sure it is set before importing torch (or at least before you do anything cuda related in torch).
* 위의 이슈에 따르면 `CUDA_VISIBLE_DEVICES` 는 드라이버가 초기화 되면서 준비되는거기 때문에 드라이버가 인식되기 전에 미리 설정해야 하는 변수임. 따라서 `torch.cuda.is_available()` 이 먼저 나오면 안됌

In [ ]:

## torch.cuda.is_available()이 먼저 드라이버가 인식이 되어서 gpu idx 설정이 불가함
# if torch.cuda.is_available() and multi_gpu:
#     # 아래 이걸 구성하면 한 jupyter에서 training, evaluation 진행이 불가함.
#     os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#     os.environ['CUDA_VISIBLE_DEVICES'] = gpu_idx


if use_gpu:
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"]= gpu_idx

    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
    
print("learning rate: %.4e" % lr)
print("batch size: %d" % batch_size)
print("number of epoch: %d" % num_epoch)
print("data dir: %s" % data_dir)
print("ckpt dir: %s" % ckpt_dir)
print("log dir: %s" % log_dir)
print("result dir: %s" % result_dir)


print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

In [ ]:
## dataset for training
transform = transforms.Compose([Normalization(mean=0.5, std=0.5), RandomFlip(), ToTensor()])

dataset_train = Dataset(data_dir=os.path.join(data_dir, 'train'), transform=transform)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)

dataset_val = Dataset(data_dir=os.path.join(data_dir, 'val'), transform=transform)
loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=0)

# 그밖에 부수적인 variables 설정하기
# tensorboard외 loss 진행사항을 출력하기 위한 변수
num_data_train = len(dataset_train)
num_data_val = len(dataset_val)

# batch로 인한 training 수를 확인하기 위한 변수
num_batch_train = np.ceil(num_data_train / batch_size)
num_batch_val = np.ceil(num_data_val / batch_size)

### 2-1. dataset visualization

In [ ]:
def tensor_to_imgnp_labelnp(img_tensor, label_tensor):
    """
    img, label : tensor -> np & (C, H, W) -> (H, W, C) & sqeeze dim
    img: + denormalizing
    """
    
    fn_tonumpy = lambda x: x.to('cpu').detach().numpy().transpose(1, 2, 0) # tensor to np & (c, h, w) -> (h, w, c)
    fn_denorm = lambda x, mean, std: (x * std) + mean    # norm val -> denormalization

    label = fn_tonumpy(label_tensor).squeeze()     # channel 삭제
    input = fn_tonumpy(fn_denorm(img_tensor, mean=0.5, std=0.5)).squeeze()
    
    return input, label

def draw_img_label(img_np, label_np, cmap='color', size=None, ratio=None, img_title=None, label_title=None):
    # cmap에 따라 컬러 or 흑백 이미지 그림
    %matplotlib inline
    
    if type(img_np) != np.ndarray:
        img_np = read_img_to_np(img_np, cmap=cmap)
        label_np = read_img_to_np(label_np, cmap=cmap)
    
    print(f"input shape: {img_np.shape}")
    print(f"label shape: {label_np.shape}")
    
    if size:
        plt.figure(figsize=size)
    elif ratio:
        plt.figure(figsize=(8*ratio, 8*ratio))
    else:
        plt.figure()
    
    
    plt.subplot(121)
    plt.imshow(img_np, cmap=cmap)
    if not img_title:
        img_title = "input"
    plt.title(img_title, fontsize=15)

    
    plt.subplot(122)
    plt.imshow(label_np, cmap=cmap)
    if not label_title:
        label_title = "label"
    plt.title(label_title, fontsize=15)
    
    plt.show()
    
    
def draw_img_label_output(
    img_np, label_np, output_np, cmap='color', size=None, ratio=None, 
    img_title=None, label_title=None, output_title=None
):
    # cmap에 따라 컬러 or 흑백 이미지 그림
    %matplotlib inline
    
    if type(img_np) != np.ndarray:
        img_np = read_img_to_np(img_np, cmap=cmap)
        label_np = read_img_to_np(label_np, cmap=cmap)
        output_np = read_img_to_np(output_np, cmap=cmap)
    
    print(f"input shape: {img_np.shape}")
    print(f"label shape: {label_np.shape}")
    print(f"output shape: {output_np.shape}")
    
    if size:
        plt.figure(figsize=size)
    elif ratio:
        plt.figure(figsize=(8*ratio, 8*ratio))
    else:
        plt.figure()
    
    
    plt.subplot(131)
    plt.imshow(img_np, cmap=cmap)
    if not img_title:
        img_title = "input"
    plt.title(img_title, fontsize=15)

    
    plt.subplot(132)
    plt.imshow(label_np, cmap=cmap)
    if not label_title:
        label_title = "label"
    plt.title(label_title, fontsize=15)
    
    plt.subplot(133)
    plt.imshow(output_np, cmap=cmap)
    if not output_title:
        output_title = "output"
    plt.title(output_title, fontsize=15)
    
    plt.show()

In [ ]:
sample = next(iter(dataset_train))
input, label = sample['input'], sample['label']
input, label = tensor_to_imgnp_labelnp(input, label)
draw_img_label(input, label, cmap='gray', size=(12, 12))

## st3. Network

In [ ]:
## 네트워크 생성하기

# multi gpu setting
if torch.cuda.is_available() and multi_gpu:
    net = UNet()
    # net = nn.DataParallel(net).to(device)
    net = nn.DataParallel(net)
    net.cuda()
else:
    net = UNet().to(device)

## 손실함수 정의하기 - binary classification
fn_loss = nn.BCEWithLogitsLoss().to(device)

## Optimizer 설정하기
optim = torch.optim.Adam(net.parameters(), lr=lr)

## 그밖에 부수적인 functions 설정하기
fn_tonumpy = lambda x: x.to('cpu').detach().numpy().transpose(0, 2, 3, 1) # tensor to np
fn_denorm = lambda x, mean, std: (x * std) + mean    # norm val -> denormalization (tensorboard 적용)
fn_class = lambda x: 1.0 * (x > 0.5)    # binary class로 변경(softmax 개념) (tensorboard 적용)

## Tensorboard 를 사용하기 위한 SummaryWriter 설정
writer_train = SummaryWriter(log_dir=os.path.join(log_dir, 'train'))
writer_val = SummaryWriter(log_dir=os.path.join(log_dir, 'val'))

net

## st4. Training

In [ ]:
VERBOSE = 2     # 1: all print / 2: epoch print / False: only time print


st_epoch = 0
epoch_time_list = list()

if train_continue == "on":
    # 특히 colab에서 session 재시작의 경우 문제가 생길 수 있으므로. 다음과 같이 설정
    net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

for epoch in range(st_epoch + 1, num_epoch + 1):
    net.train()
    loss_arr = []

    print('-------------- epoch {} ----------------'.format(epoch)) 
    since = time.time() 
    
    for batch, data in enumerate(loader_train, 1):
        # forward pass
        label = data['label'].to(device)
        input = data['input'].to(device)

        output = net(input)

        # backward pass
        optim.zero_grad()

        loss = fn_loss(output, label)
        loss.backward()

        optim.step()

        # 손실함수 계산
        # 내부 format 궁금하네
        loss_arr += [loss.item()]

        if VERBOSE == 1:
            print("TRAIN: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
                  (epoch, num_epoch, batch, num_batch_train, np.mean(loss_arr)))

        # Tensorboard 저장하기
        label = fn_tonumpy(label)
        input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
        output = fn_tonumpy(fn_class(output))

        # Tensorboard 에 img를 저장
        writer_train.add_image('label', label, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('input', input, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('output', output, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')

    # epoch당 Tensorboard에 loss 저장
    writer_train.add_scalar('loss', np.mean(loss_arr), epoch)

    # epoch당 validation 시작
    with torch.no_grad():   # backpropa block
        net.eval()          # network에 validation 명시
        loss_arr_val = []

        for batch, data in enumerate(loader_val, 1):
            # forward pass
            label = data['label'].to(device)
            input = data['input'].to(device)

            output = net(input)

            # 손실함수 계산하기
            loss = fn_loss(output, label)

            loss_arr_val += [loss.item()]

            if VERBOSE == 1:
                print("VALID: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
                      (epoch, num_epoch, batch, num_batch_val, np.mean(loss_arr)))

            # Tensorboard 저장하기
            label = fn_tonumpy(label)
            input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
            output = fn_tonumpy(fn_class(output))

            writer_val.add_image('label', label, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('input', input, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('output', output, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')

    writer_val.add_scalar('loss', np.mean(loss_arr_val), epoch)
    
    if VERBOSE:
        print(f"EPOCH {epoch} / {num_epoch} | TRAIN mean.LOSS {np.mean(loss_arr):.3f}   VAL: mean.LOSS {np.mean(loss_arr_val):.3f}")
    
    time_elapsed = time.time() - since  
    print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    epoch_time_list.append(time_elapsed)

    if epoch % 50 == 0:
        # 50 epoch당 모델 저장
        save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch)

# 학습 완료시 tensorboard closing
writer_train.close()
writer_val.close()

In [ ]:
# 시간 측정 테이블 저장

time_result_dir = "learning_time"
out_fname = 'time_exp3.csv'
if not os.path.exists(time_result_dir):
    os.makedirs(time_result_dir)

df = pd.DataFrame(epoch_time_list, columns=["vanillaUNet_model"])
file_name = os.path.join(time_result_dir, out_fname)
df.to_csv(file_name, index=False)
df.head()

## st5. Testing / Evaluation

In [ ]:
## result dir 생성하기
if not os.path.exists(result_dir):
    os.makedirs(os.path.join(result_dir, 'png'))
    os.makedirs(os.path.join(result_dir, 'numpy'))

# dataset for validation
transform = transforms.Compose([Normalization(mean=0.5, std=0.5), ToTensor()])

dataset_test = Dataset(data_dir=os.path.join(data_dir, 'test'), transform=transform)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=0)

# 그밖에 부수적인 variables 설정하기
num_data_test = len(dataset_test)

num_batch_test = np.ceil(num_data_test / batch_size)

In [ ]:
device

In [ ]:
def load(ckpt_dir, net, optim, cuda_idx=None):
    print("!!!!!!!")
    if not os.path.exists(ckpt_dir):
        # ckpt 없다면 기본 셋팅 return
        epoch = 0
        return net, optim, epoch

    # ckpt있다면 학습 마지막 모델 load
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    if cuda_idx == None:
        dict_model = torch.load('%s/%s' % (ckpt_dir, ckpt_lst[-1]))
    else:
        dict_model = torch.load('%s/%s' % (ckpt_dir, ckpt_lst[-1]), map_location='cuda:0')

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])

    return net, optim, epoch

In [ ]:
# TEST MODE (맨 마지막 model을 불러와서 evaluation 진행)
net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

with torch.no_grad():
    net.eval()
    loss_arr = []

    for batch, data in enumerate(loader_test, 1):
        # forward pass
        label = data['label'].to(device)
        input = data['input'].to(device)

        output = net(input)

        # 손실함수 계산하기
        loss = fn_loss(output, label)

        loss_arr += [loss.item()]

        print("TEST: BATCH %04d / %04d | LOSS %.4f" %
              (batch, num_batch_test, np.mean(loss_arr)))

        # Tensorboard 저장하기
        label = fn_tonumpy(label)
        input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
        output = fn_tonumpy(fn_class(output))

        for j in range(label.shape[0]):
            id = num_batch_test * (batch - 1) + j

            plt.imsave(os.path.join(result_dir, 'png', 'label_%04d.png' % id), label[j].squeeze(), cmap='gray')
            plt.imsave(os.path.join(result_dir, 'png', 'input_%04d.png' % id), input[j].squeeze(), cmap='gray')
            plt.imsave(os.path.join(result_dir, 'png', 'output_%04d.png' % id), output[j].squeeze(), cmap='gray')

            np.save(os.path.join(result_dir, 'numpy', 'label_%04d.npy' % id), label[j].squeeze())
            np.save(os.path.join(result_dir, 'numpy', 'input_%04d.npy' % id), input[j].squeeze())
            np.save(os.path.join(result_dir, 'numpy', 'output_%04d.npy' % id), output[j].squeeze())

print("AVERAGE TEST: BATCH %04d / %04d | LOSS %.4f" %
      (batch, num_batch_test, np.mean(loss_arr)))

### 5-1 dataset visualization

In [ ]:
sample = next(iter(dataset_test))
input, label = sample['input'], sample['label']
input = input.to(device).unsqueeze(dim=0)  # B dim add

output = net(input)

output = fn_tonumpy(fn_class(output))
output = output.squeeze()
input = input.squeeze(dim=0)

input, label = tensor_to_imgnp_labelnp(input, label)
draw_img_label_output(input, label, output, cmap='gray', size=(17, 6))

# st 6. tensorboard check
* train, validation 만 저장함

## 6-1 이미지 수 계산

In [ ]:
train_img_cnt = int(num_batch_train * num_epoch)
val_img_cnt = int(num_batch_val * num_epoch)

print(f"train batch 수: {num_batch_train} / val batch 수: {num_batch_val}")
print(f"loss 수: {num_epoch} epoch")
print(f"tenboard train imgs : {train_img_cnt} 장 / tb val imgs : {val_img_cnt} 장")

In [ ]:
%load_ext tensorboard

In [ ]:
# tensorboard --host=0.0.0.0 --logdir='/data/kehyeong/project/youtube-cnn-002-pytorch-unet/out/log'

# st Opt. remove output

In [ ]:
# import shutil

# out_dir = './out'
# shutil.rmtree(out_dir)